In [4]:

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3,4,5,6,7'
#os.environ['CUDA_VISIBLE_DEVICES'] = '4,5,6,7'
import torch




# 确保PyTorch能够检测到GPU

In [5]:

import json
import time
import re
import itertools
import numpy as np
import pandas as pd
import datetime as DT
from tqdm import tqdm
from datetime import datetime
from scipy.optimize import curve_fit
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer

In [6]:

# Load pretrained LLM
QWEN2d5_7B_INST = "/data/models/hub/models--Qwen--Qwen2.5-7B-Instruct/snapshots/acbd96531cda22292a3ceaa67e984955d3965282"
QWEN2d5_32B_INST_AWQ = "/home/peizhengqi/Qwen/Qwen2.5-32B-Instruct-AWQ"
QWEN2d5_72B_INST_AWQ = "/data/models/hub/models--Qwen--Qwen2.5-72B-Instruct-AWQ/snapshots/035adeffb861e8ee98344e2c7d8f50fe585e373d"

#model_7Bn = AutoModelForCausalLM.from_pretrained(QWEN2d5_7B_INST, device_map="auto")
#tokenizer_7Bn = AutoTokenizer.from_pretrained(QWEN2d5_7B_INST, device_map="auto")

model_32Bn = AutoModelForCausalLM.from_pretrained(QWEN2d5_32B_INST_AWQ, device_map="auto")
tokenizer_32Bn = AutoTokenizer.from_pretrained(QWEN2d5_32B_INST_AWQ, device_map="auto")

#model_72Bn = AutoModelForCausalLM.from_pretrained(QWEN2d5_72B_INST_AWQ, device_map="auto")
#tokenizer_72Bn = AutoTokenizer.from_pretrained(QWEN2d5_72B_INST_AWQ, device_map="auto")

Loading checkpoint shards: 100%|██████████| 5/5 [00:05<00:00,  1.10s/it]


In [7]:

from transformers import TextStreamer

streamer = TextStreamer(tokenizer_32Bn, skip_prompt=True, skip_special_tokens=True)

In [8]:

sys_prompt1 = """
    You are a helpful, respectful and honest assistant. 
    Always answer as helpfully as possible, while being safe. 
    Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
    Please ensure that your responses are socially unbiased and positive in nature.
    If you don't know the answer to a question, please don't share false information.
"""


def message_to_response_via_LLM(model, tokenizer, messages, max_new_tokens=256):

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.01, streamer=streamer)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [9]:
raw_excel = pd.DataFrame()
for file in os.listdir('data'):
    if 'csv' in file:
        df = pd.read_csv(f'data/{file}')
        raw_excel = pd.concat([df, raw_excel], axis=0)

In [10]:
raw_excel = raw_excel.rename(columns={'sessionId': 'chatRoomId', '内容': 'text'})
raw_excel

,chatRoomId,日期,senderId,发送时间,senderType,信息类型,text
0,3sQb80ObARjiFKq,2020-06-19,5d0b4dea91d90703a7bcc752,2020-06-19 12:18:02.124,患者,卡片,NaN
1,3sQb80ObARjiFKq,2020-06-19,5cf8c33fced4a17773838cdf,2020-06-19 13:33:39.009,照护师,卡片,NaN
2,3sQb80ObARjiFKq,2020-06-19,5d0b4dea91d90703a7bcc752,2020-06-19 13:45:11.154,患者,文字,有牛肉，拌在菜里了
3,3sQb80ObARjiFKq,2020-06-19,a56be247185bb1992f7fd34f,2020-06-19 13:48:41.016,照护师,文字,好的😄
4,3sQb80ObARjiFKq,2020-06-21,66728d10dc75bc6a43052036,2020-06-21 08:59:37.185,系统,提示信息,上传了早餐
...,...,...,...,...,...,...,...
75275,FIUrkExDLGvZGMT,2024-01-25,66728d10dc75bc6a43052036,2024-01-25 18:01:23.363,系统,提示信息,上传了晚餐
75276,FIUrkExDLGvZGMT,2024-01-25,65b062e2305df794ce0b2f77,2024-01-25 18:01:23.380,患者,卡片,NaN
75277,FIUrkExDLGvZGMT,2024-01-25,65092ea1ac6a7d3a507c1ec2,2024-01-25 18:53:24.035,照护师,卡片,NaN
75278,FIUrkExDLGvZGMT,2024-01-25,65092ea1ac6a7d3a507c1ec2,2024-01-25 18:53:48.949,照护师,卡片,NaN


In [11]:

user_list = list(set(raw_excel['chatRoomId']))


In [12]:

def get_QA_text(text_list, user_list):
    QA_list = []
    for i in range(len(text_list)):
        if user_list[i] == '患者':
            QA_list.append('患者：' + str(text_list[i]))
        elif user_list[i] == '系统':
            QA_list.append('系统：' + str(text_list[i]))
        else:
            QA_list.append('医生：' + str(text_list[i]))
    return '\n'.join(QA_list)

In [13]:


filtered_df = raw_excel[raw_excel['chatRoomId'] == "002sC9RqpqqW8pa"]
QA_text = get_QA_text(list(filtered_df['text']), list(filtered_df['senderType']))
#filtered_df

QA_text.split('\n')


['患者：需要血糖仪',
 '患者：收件地址：北京市西城区小马厂路一号院 西豪逸景七号楼506。 收件人：姚裕群。手机13141329125。 ',
 '医生：收到',
 '患者：大夫，我2月9日早上没有测血糖，可能是晚上测血糖时按错了地方，请帮助删掉。(数据是晚上的.我已填上)。',
 '患者：以下请转李大夫：.李大夫好。我日前看病，因骨质疏松开了骨化三醇软胶囊和鲑鱼降钙素喷鼻剂，门诊时问了李大夫，不影响降糖药的使用。刚看了说明，骨化三醇还影响甲状腺功能。我做了甲狀腺次全切，一直服优甲乐。现在吃骨化三醇，优甲乐是否应当减量吃？还有，骨化三醇与优甲乐同时吃，应当多长时间测甲功？谢谢。姚裕群    ',
 '医生：您好，骨化三醇不影响甲功。优甲乐不用减量。常规半年测一次就行',
 '患者：好的。谢谢！姚裕群',
 '医生：不客气',
 '医生：还有5天您就要来就诊了，但我们还没有收到您近7天的血糖监测数据，医生有可能无法对您的情况作更全面的判断和针对性的指导，请您抓紧测量',
 '医生：您已于11:40测量了午餐前血糖，从吃第一口饭算起，您应该测量餐后血糖了。配对监测有助于医生了解您饮食与药物搭配是否合理，请遵医嘱测量',
 '医生：招募公告：北京电视台科教频道《健康北京》栏目，7月24日（周二）16:00-17:00在北京电视台国贸台址（东三环外）演播楼，进行“糖尿病运动指导”节目录制，主讲嘉宾为北京大学第一医院临床营养科主任郭晓蕙，国家体育总局体育科学研究所体医融合中心主任郭建军。将就糖尿病人的运动强度及餐前低血糖、运动前的风险评估、餐前高强度运动+餐后运动、运动量及饮食用药调整，进行讲解和指导。现征集30名现场观众，于7月24日（周二）15:20在北京电视台国贸台址南门（通惠河北路路北）集合进场，录制当天现场提供瓶装矿泉水。截至报名时间：2018年7月20日',
 '医生：您已于18:24测量了晚餐前血糖，从吃第一口饭算起，您应该测量餐后血糖了。配对监测有助于医生了解您饮食与药物搭配是否合理，请遵医嘱测量',
 '医生：您刚才测量的餐后血糖是15.9，餐前血糖是6.3，相差比较大，您能大概描述下您刚吃的是什么吗？',
 '患者：这个数字比较意外。总体上比较注意控制饮食。今天晚饭吃的东西一般，主食是米饭，约二两，菜吃的多些，扁豆和土豆。没喝粥等升糖快的食物。',


In [14]:

len(QA_text.split('\n'))
chat_str = "\n".join(QA_text.split('\n')[:500])
len(chat_str)


6908

In [15]:

rule_templates = [
    "一日三餐前后的血糖波动范围应该控制在[]mmol/L以内。",
]

In [16]:

query_prompt = "请根据该组聊天记录对规则模版库的内容进行填充，请将具体内容填充至[]中，得到完整的规则并输出即可，无需解释推导过程。"
query_prompt = "请从该组聊天记录提取有价值的规则，例如「一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内」。"


bg_prompt = "已知今天的日期是：{}，北京时间。".format(time.asctime(time.localtime()))
#format_prompt = "并将最终得到的规则们整理成list的形式作为输出。"
format_prompt = "并将最终得到的规则们整理成list的形式作为输出，仅仅输出rules中的内容"


res_messages = [
    {"role": "system", "content": sys_prompt1},
    {"role": "user", "content": "给定一组聊天记录数据："+chat_str},
    #{"role": "user", "content": "给定一个规则模版库："+str(rule_templates)},
    {"role": "user", "content": query_prompt+format_prompt}
]

In [17]:

llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)

llm_response

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


[
    "一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内",
    "土豆、白薯、芋头等食物的摄入量应控制在每餐不超过2两",
    "早餐应在早上8点前完成，并且要包含主食、蔬菜和肉蛋类",
    "餐后2小时应测量血糖，以便了解饮食与药物搭配是否合理",
    "睡前血糖若小于6，可适当加餐，否则应避免加餐",
    "每餐主食不宜过多，最好控制在2两以内，且应包含多种类型的食物",
    "避免食用升糖快的食物，如粥、甜品等",
    "每日应定时定量服药，并按时测量血糖",
    "适量运动有助于控制血糖，餐后应适当增加运动量"
]


'[\n    "一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内",\n    "土豆、白薯、芋头等食物的摄入量应控制在每餐不超过2两",\n    "早餐应在早上8点前完成，并且要包含主食、蔬菜和肉蛋类",\n    "餐后2小时应测量血糖，以便了解饮食与药物搭配是否合理",\n    "睡前血糖若小于6，可适当加餐，否则应避免加餐",\n    "每餐主食不宜过多，最好控制在2两以内，且应包含多种类型的食物",\n    "避免食用升糖快的食物，如粥、甜品等",\n    "每日应定时定量服药，并按时测量血糖",\n    "适量运动有助于控制血糖，餐后应适当增加运动量"\n]'

#统计所有规则

In [23]:
all_rules_list = []
other_output = []
for i in tqdm(range(len(user_list))):
    print(i)
    if i >=1147:
        

        filtered_df = raw_excel[raw_excel['chatRoomId'] == user_list[i]]
        QA_text = get_QA_text(list(filtered_df['text']), list(filtered_df['senderType']))
        chat_str = "\n".join(QA_text.split('\n')[:500])
        res_messages = [
        {"role": "system", "content": sys_prompt1},
        {"role": "user", "content": "给定一组聊天记录数据："+chat_str},
        #{"role": "user", "content": "给定一个规则模版库："+str(rule_templates)},
        {"role": "user", "content": query_prompt+format_prompt}
    ]
        llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)

        try:
            rules_list = llm_response.split('[')[1].split(']')[0].split(',')
            all_rules_list += rules_list
        except:
            other_output.append(llm_response)
    

  0%|          | 0/11295 [00:00<?, ?it/s]

system

    You are a helpful, respectful and honest assistant. 
    Always answer as helpfully as possible, while being safe. 
    Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
    Please ensure that your responses are socially unbiased and positive in nature.
    If you don't know the answer to a question, please don't share false information.

user
给定一组聊天记录数据：医生：齐秀荣，您好，欢迎您加入共同照护，您只需要定期监测血糖，上传饮食，保持联系，我们将为您提供健康咨询、血糖监督、饮食指导、知识推送等服务，您在血糖管理中遇到任何疑问，都可以点击“问医生”，向我们寻求专业的指导和建议，您的血糖健康，我们共同守护！
系统：nan
系统：08-15 15:52 (午餐后 13.3)
系统：nan
医生：nan
医生：nan
医生：您最近都没有上传饮食，健康的饮食是稳定血糖的关键，上传饮食后我会及时给您评价，帮您控好血糖，记得上传饮食哦~
系统：上传了晚餐加餐
患者：nan
系统：nan
系统：nan
患者：nan
系统：08-16 06:24 (早餐前 7.5)
患者：nan
系统：上传了早餐
患者：nan
医生：nan
医生：您好，早餐蔬菜多样化，清淡量适宜，不错，牛奶含丰富蛋白质，量适宜，很棒，主食量适宜，整体饮食结构搭配合理，营养均衡。
医生：nan
医生：nan
系统：08-16 08:58 (早餐前 10.7)
医生：您刚刚测的餐前血糖是10.7，请您及时上传本餐饮食，如有不合理地方，我们会帮您调整饮食方案，避免血糖波动过大
系统：上传了午餐
患者：nan
医生：nan
医生：注意餐前餐后配对监测哈。
患者：我尿酸高，医生说现在不让我吃粗粮
医生：您好，是尿ACr，暂时不吃豆制品及

 10%|█         | 1148/11295 [00:19<02:49, 59.74it/s]

[
    "坚持定期测量血压，以便医生了解血压情况并提供针对性指导",
    "血糖波动较大时（如餐前6.4mmol/L，餐后11.4mmol/L），需要确认是否按医嘱用药",
    "饮酒需适量，即使偶尔应酬性饮酒也应注意控制量",
    "如果条件允许，每周应监测3次餐前餐后的血糖值，包括早餐、午餐和晚餐各一次",
    "使用血糖仪

 10%|█         | 1148/11295 [00:29<02:49, 59.74it/s]

...

[
    "餐后2小时血糖的理想值应控制在7.1 mmol/L或以下",
    "加餐时间建议安排在上午9点左右，并且距离下一餐至少2小时以上",
    "午餐时应确保足够的蛋白质摄入，以避免下午饥饿",
    "注意血糖仪和试纸的工作温度应在10~40度之间，以保证血糖数值的准确性",
    "保持血糖平稳，避免血糖波动过大"
]


 50%|████▉     | 5617/11295 [13:01:05<20:19:40, 12.89s/it]

```python
[
    "患者的控糖目标为4-7 mmol/L",
    "患者在07月20日05:54和07:58两次空腹血糖测量均达标，分别为6.7 mmol/L和5.7 mmol/L",
    "患者提到的餐2显示为空腹状态，需要进一步确认是否为测量错误或特殊情况",
    "医生对患者的测量记录进行了修正"
]
```


 50%|████▉     | 5618/11295 [13:01:14<18:45:20, 11.89s/it]

[
"主食和蔬菜搭配不错，但应避免摄入油脂含量较高的食物如小鱼，因其对血糖影响较大",
"晚餐控制好量，尤其是汤类，应减少摄入",
"餐后注意适当活动，有助于平稳血糖",
"晚餐蔬菜量应充足，但主食不宜过多，不超过2两",
"沙格列汀二甲双胍缓释片应在早上饭后服用，盐酸二甲双胍片应在晚上饭后服用",
"用药顺序需要

In [17]:
len(set(user_list))

11295

In [22]:
# 定义要写入的规则

# 打开或创建一个名为 'rules.txt' 的文件，以写入模式 ('w') 打开
with open('all_rules_28.txt', 'w') as file:
    # 遍历规则列表，并将每条规则写入文件
    for rule in all_rules_list:
        file.write(rule + '\n')  # 添加换行符以便每条规则占一行
print("Rules have been written to 'rules.txt'")


Rules have been written to 'rules.txt'


In [ ]:
len(''.join(all_rules_list))

In [ ]:
chat_str = str(all_rules_list[:550])

query_prompt = "请根据该组聊天记录对规则模版库的内容进行填充，请将具体内容填充至[]中，得到完整的规则并输出即可，无需解释推导过程。"
query_prompt = "请从该组聊天记录提取有价值的规则，例如「一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内」。"


bg_prompt = "已知今天的日期是：{}，北京时间。".format(time.asctime(time.localtime()))
#format_prompt = "并将最终得到的规则们整理成list的形式作为输出。"
format_prompt = "请将它们进行语义去重，并以list的形式输出"


res_messages = [
    {"role": "system", "content": sys_prompt1},
    {"role": "user", "content": "给定一组规则list："+chat_str},
    #{"role": "user", "content": "给定一个规则模版库："+str(rule_templates)},
    {"role": "user", "content": query_prompt+format_prompt}
]
llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)
rules_list1 = llm_response.split('[')[1].split(']')[0].split(',')


In [ ]:
chat_str = str(all_rules_list[:100])
char_shot = str(['\n    "一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内"',
 '\n    "餐后血糖最好能控制在10mmol/L以下"',
 '\n    "患者需要定期监测餐前和餐后血糖，并上传饮食记录以便获得专业指导"',
 '\n    "患者应按医嘱用药，并注意饮食控制，必要时可适当增加餐后运动"',
 '\n    "患者若出现低血糖症状（如出汗、心慌）应及时测量血糖并采取相应措施"',
 '\n    "患者应保持与医生的定期联系，及时反馈血糖监测结果和身体状况"\n',
 '"餐前血糖应控制在7.0mmol/L以下"',
 ' "餐后血糖应控制在10.0mmol/L以下"',
 ' "餐前餐后血糖波动范围应控制在3.5mmol/L以内"',
 ' "不建议主食用水果代替"',
 ' "建议每日监测餐前餐后血糖"',
 ' "运动量不宜过大，避免引起身体不适"',
 ' "饮食中应包含足够的蔬菜和蛋白质"',
 ' "避免食用油炸食品和高糖食品"',
 ' "定期上传饮食图片以便医生进行指导"',
 '\n    "一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内"',])
assistant_an = str(['\n    "一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内"',
 '\n    "餐后血糖最好能控制在10mmol/L以下"',
 '\n    "患者需要定期监测餐前和餐后血糖，并上传饮食记录以便获得专业指导"',
 '\n    "患者应按医嘱用药，并注意饮食控制，必要时可适当增加餐后运动"',
 '\n    "患者若出现低血糖症状（如出汗、心慌）应及时测量血糖并采取相应措施"',
 '\n    "患者应保持与医生的定期联系，及时反馈血糖监测结果和身体状况"\n',
 '"餐前血糖应控制在7.0mmol/L以下"',
 ' "不建议主食用水果代替"',
 ' "建议每日监测餐前餐后血糖"',
 ' "运动量不宜过大，避免引起身体不适"',
 ' "饮食中应包含足够的蔬菜和蛋白质"',
 ' "避免食用油炸食品和高糖食品"',
 ' "定期上传饮食图片以便医生进行指导"',
 ])
query_prompt = "请根据该组聊天记录对规则模版库的内容进行填充，请将具体内容填充至[]中，得到完整的规则并输出即可，无需解释推导过程。"
query_prompt = "请从该组聊天记录提取有价值的规则，例如「一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内」。"


bg_prompt = "已知今天的日期是：{}，北京时间。".format(time.asctime(time.localtime()))
#format_prompt = "并将最终得到的规则们整理成list的形式作为输出。"
format_prompt = "请将它们进行语义去重，并以list的形式输出"


res_messages = [
    {"role": "system", "content": sys_prompt1},
    {"role": "user", "content": "给定一组规则list："+chat_str},
    {"role": "assistant", "content": assistant_an},
    {"role": "user", "content": "给定一组规则list："+chat_str},
    #{"role": "user", "content": "给定一个规则模版库："+str(rule_templates)},
    {"role": "user", "content": query_prompt+format_prompt}
]
llm_response = message_to_response_via_LLM(model_32Bn, tokenizer_32Bn, res_messages, max_new_tokens=1000)
rules_list_ = llm_response.split('[')[1].split(']')[0].split(',')

In [ ]:
len(rules_list1)

In [60]:
# 定义一个包含文本的列表
text_list = ["这是第一行文本", "这是第二行文本", "这是第三行文本"]

# 打开一个文件用于写入（如果文件不存在会自动创建）
with open("rules.txt", "w", encoding="utf-8") as file:
    # 遍历列表中的每一个元素
    for text in all_rules_list:
        # 将每个元素写入文件，并在每个元素后添加换行符
        file.write(text + "\n")


In [81]:
# 使用内置的 open() 函数来读取文件
file_path = 'rules.txt'  # 替换为你的记事本文件路径

with open(file_path, 'r', encoding='utf-8') as file:
    content_list = file.readlines()

In [82]:
content_list = [ele.strip() for ele in content_list if len(ele) > 3]

In [83]:
content_list

['"一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内"',
 '"运动可以增加胰岛素的敏感性，有利于餐后血糖控制"',
 '"睡前不低于6点尽量不吃东西，建议测量晚睡前和次日空腹血糖以分析空腹血糖偏高的原因"',
 '"如果某一天计划减少运动量，相对应的餐也要稍加控制，才能保证血糖稳定"',
 '"早餐搭配蔬菜很好，可以测量晚餐前后血糖以观察血糖情况"',
 '"晚餐后至睡前这段时间尽量不要吃东西"',
 '"粗细单配制作饺子皮，素馅为主，适量吃饺子并搭配蔬菜，避免蘸高热量酱料"',
 '"持续监测餐前餐后血糖，如有不合理地方及时调整饮食方案"',
 '"一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内"',
 '"餐后血糖最好能控制在10mmol/L以下"',
 '"患者需要定期监测餐前和餐后血糖，并上传饮食记录以便获得专业指导"',
 '"患者应按医嘱用药，并注意饮食控制，必要时可适当增加餐后运动"',
 '"患者若出现低血糖症状（如出汗、心慌）应及时测量血糖并采取相应措施"',
 '"患者应保持与医生的定期联系，及时反馈血糖监测结果和身体状况"',
 '"餐前血糖应控制在7.0mmol/L以下"',
 '"餐后血糖应控制在10.0mmol/L以下"',
 '"餐前餐后血糖波动范围应控制在3.5mmol/L以内"',
 '"不建议主食用水果代替"',
 '"建议每日监测餐前餐后血糖"',
 '"运动量不宜过大，避免引起身体不适"',
 '"饮食中应包含足够的蔬菜和蛋白质"',
 '"避免食用油炸食品和高糖食品"',
 '"定期上传饮食图片以便医生进行指导"',
 '"一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内"',
 '"餐后血糖控制在10.0mmol/l以内是可以加些水果的，相应的主食可以减少一部分，避免热量摄入过多"',
 '"出现低血糖后，尽量优先选择糖分高、好吸收的食物，如一勺白砂糖、一勺蜂蜜或者沏到水里喝，如果家里有方糖或葡萄糖片的话，可以吃2-3片，大约15g左右，过15分钟后在复测一下血糖"',
 '"每餐主食的量应该控制在2两以内，避免因主食过量造成的大波动"',
 '"建议每餐都要吃蔬菜，早餐如果来不及炒菜，可以直接吃一根黄瓜或是一个大西红柿"',
 '"建议餐前＋餐后2小时血糖配对监测，这样更利于观察血糖

In [20]:
len(all_rules_list)

426

In [76]:
'    "一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内"\n'.strip()

'"一日三餐前后的血糖波动范围应该控制在3.5mmol/L以内"'